# Analyze the JS divergences

## Preamble

In [11]:
import psutil
p = psutil.Process()
p.cpu_affinity([0])

import numpy as np 
import matplotlib.pyplot as plt
import pickle
from IPython.display import display, Math
import pandas as pd

Utilities:

In [12]:
parameters_str = [r"$\mathcal{M}$",
                  r"$q$",
                  r"$\chi_1$",
                  r"$\chi_2$",
                  r"$\Lambda_1$",
                  r"$\Lambda_2$",
                  r"$d_L$",
                  r"$\phi_c$",
                  r"$\cos \iota$",
                  r"$\psi$",
                  r"$\alpha$",
                  r"$\sin \delta$",
]

In [13]:
get_path = lambda x: f"../figures/{x}_js_div.pkl"

In [14]:
def get_js_table(events_list: list[str],
                 nb_round: int = 6,
                 filler_str: str = "0") -> pd.DataFrame:
    """
    Function to create the JS divergence table by fetching the values from the respective run dirs.

    Args:
        events_list (list[str]): List of events to analyze, e.g. ["GW190412_TaylorF2"]

    Returns:
        pd.DataFrame: Dataframe containing the events and the JS divergences for all parameters
    """
    
    table = {"Parameter": parameters_str}
    MY_LENGTH = nb_round + 2 # number of characters, including the "0." hence the +2
    
    for event in events_list:
        # Load the JS values
        path = get_path(event)
        with open(path, "rb") as f:
            js_div_dict = pickle.load(f)
            
        # Convert and fetch the highest value's index
        js_div_values = np.array(list(js_div_dict.values()))
        js_div_values = np.round(js_div_values, nb_round)
        max_index = np.argmax(js_div_values)
        
        # Get their string representations
        js_div = [str(value) for value in js_div_values]
        for i, value in enumerate(js_div):
            length_difference = MY_LENGTH - len(value)
            if length_difference > 0:
                js_div[i] = value + filler_str * (MY_LENGTH - len(value))
        
        js_div = [f"${value}$" for value in js_div]
        
        # Make the highest value bold
        js_div[max_index] = "$\\bm{{{}}}$".format(str(js_div_values[max_index]))
        table[event] = js_div
    
    print("table")
    print(table)
    
    table = pd.DataFrame.from_dict(table)
    table.rename(columns={'GW170817_TaylorF2': 'TF2', 
                       'GW170817_NRTidalv2': 'NRTv2',
                       'GW190425_TaylorF2': 'TF2', 
                       'GW190425_NRTidalv2': 'NRTv2',}, inplace=True)
    return table

In [15]:
def my_to_latex(df: pd.DataFrame, 
                extra_table_header: str = None,
                column_format: str = None,
                only_tabular: bool = True,
                use_hline: bool = True) -> str:
    """ 
    Own auxiliary function to create latex code of tables in pandas DF format.
    """
    
    n_cols = len(df.columns)
    
    # Column format
    if column_format is None:
        column_format = "l " + " c " * (n_cols - 1)
    
    result = df.style \
        .hide(axis="index")\
        .to_latex(position_float="centering", hrules=True, column_format = column_format)
        
    # Insert "\\midrule" after the row with "ref" in the second column
    latex_table_lines = result.split('\n')
    # Find index with ref isn it:
    insert_idx = 0
    for i in range(len(latex_table_lines)):
        line = latex_table_lines[i]
        if "ref" in line:
            insert_idx = i
            break
        
    latex_table_lines.insert(insert_idx + 1, "\\midrule")
    result = '\n'.join(latex_table_lines)
    
    # Add extra line to table header, eg to group different columns
    if extra_table_header is not None:
        result = result.replace("\\toprule", "\\toprule\n" + extra_table_header)
        
    # Remove table env
    if only_tabular:
        latex_table_lines = result.split('\n')
        latex_table_lines = [line for line in latex_table_lines if "{table}" not in line and "centering" not in line]
        result = '\n'.join(latex_table_lines)
        
    if use_hline:
        result = result.replace("\\bottomrule", "\hline\hline")
        result = result.replace("\\toprule", "\hline\hline")
        result = result.replace("\midrule", "")
    
    return result 

## Make the JS divergence table

In [16]:
events_list = ["GW170817_TaylorF2",
               "GW170817_NRTidalv2",
               "GW190425_TaylorF2",
               "GW190425_NRTidalv2"]

column_names = []

In [17]:
table = get_js_table(events_list)
table

table
{'Parameter': ['$\\mathcal{M}$', '$q$', '$\\chi_1$', '$\\chi_2$', '$\\Lambda_1$', '$\\Lambda_2$', '$d_L$', '$\\phi_c$', '$\\cos \\iota$', '$\\psi$', '$\\alpha$', '$\\sin \\delta$'], 'GW170817_TaylorF2': ['$0.001725$', '$0.005212$', '$0.005633$', '$0.003030$', '$0.001062$', '$0.000559$', '$0.001544$', '$0.003500$', '$0.001615$', '$0.004048$', '$\\bm{0.014008}$', '$0.009570$'], 'GW170817_NRTidalv2': ['$0.000516$', '$0.007894$', '$0.004301$', '$0.002671$', '$0.002208$', '$0.002186$', '$\\bm{0.01847}$', '$0.010714$', '$0.012851$', '$0.011036$', '$0.001258$', '$0.001761$'], 'GW190425_TaylorF2': ['$0.003557$', '$0.004837$', '$0.002794$', '$0.002416$', '$0.008556$', '$0.005808$', '$0.001273$', '$0.003338$', '$0.006400$', '$0.001516$', '$\\bm{0.009822}$', '$0.008934$'], 'GW190425_NRTidalv2': ['$0.001884$', '$0.003883$', '$0.001904$', '$0.001400$', '$0.000652$', '$0.001450$', '$0.009157$', '$0.007874$', '$0.009502$', '$0.005820$', '$\\bm{0.014136}$', '$0.006362$']}


,Parameter,TF2,NRTv2,TF2,NRTv2
0,$\mathcal{M}$,$0.001725$,$0.000516$,$0.003557$,$0.001884$
1,$q$,$0.005212$,$0.007894$,$0.004837$,$0.003883$
2,$\chi_1$,$0.005633$,$0.004301$,$0.002794$,$0.001904$
3,$\chi_2$,$0.003030$,$0.002671$,$0.002416$,$0.001400$
4,$\Lambda_1$,$0.001062$,$0.002208$,$0.008556$,$0.000652$
5,$\Lambda_2$,$0.000559$,$0.002186$,$0.005808$,$0.001450$
6,$d_L$,$0.001544$,$\bm{0.01847}$,$0.001273$,$0.009157$
7,$\phi_c$,$0.003500$,$0.010714$,$0.003338$,$0.007874$
8,$\cos \iota$,$0.001615$,$0.012851$,$0.006400$,$0.009502$
9,$\psi$,$0.004048$,$0.011036$,$0.001516$,$0.005820$


In [18]:
# TODO: this is when all runs are finished
# table_header = "\\begin{{tabular}}{{l c c c c}}\n \hline \hline\n & \\multicolumn{{2}}{{ c }}{{GW170817}} &  \\multicolumn{{2}}{{ c }}{{GW170817}} \\\\\n"

# table_header = "\\begin{{tabular}}{{l c c c c}}\n \hline \hline\n & \\multicolumn{{2}}{{ c }}{{GW170817}} \\\\\n"

In [19]:
table_header = "& \multicolumn{2}{ c }{GW170817} & \multicolumn{2}{ c }{GW190425} \\\\\n"
table_str = my_to_latex(table, extra_table_header=table_header)

print(table_str)


\begin{tabular}{l  c  c  c  c}
\hline\hline
& \multicolumn{2}{ c }{GW170817} & \multicolumn{2}{ c }{GW190425} \\

Parameter & TF2 & NRTv2 & TF2 & NRTv2 \\

$\mathcal{M}$ & $0.001725$ & $0.000516$ & $0.003557$ & $0.001884$ \\
$q$ & $0.005212$ & $0.007894$ & $0.004837$ & $0.003883$ \\
$\chi_1$ & $0.005633$ & $0.004301$ & $0.002794$ & $0.001904$ \\
$\chi_2$ & $0.003030$ & $0.002671$ & $0.002416$ & $0.001400$ \\
$\Lambda_1$ & $0.001062$ & $0.002208$ & $0.008556$ & $0.000652$ \\
$\Lambda_2$ & $0.000559$ & $0.002186$ & $0.005808$ & $0.001450$ \\
$d_L$ & $0.001544$ & $\bm{0.01847}$ & $0.001273$ & $0.009157$ \\
$\phi_c$ & $0.003500$ & $0.010714$ & $0.003338$ & $0.007874$ \\
$\cos \iota$ & $0.001615$ & $0.012851$ & $0.006400$ & $0.009502$ \\
$\psi$ & $0.004048$ & $0.011036$ & $0.001516$ & $0.005820$ \\
$\alpha$ & $\bm{0.014008}$ & $0.001258$ & $\bm{0.009822}$ & $\bm{0.014136}$ \\
$\sin \delta$ & $0.009570$ & $0.001761$ & $0.008934$ & $0.006362$ \\
\hline\hline
\end{tabular}



## Update referee report; incorporate variations from noise in comparison

In [21]:
noise_file = "/home/thibeau.wouters/TurboPE-BNS/postprocessing/js_div_noise.npz"
noise_data = np.load(noise_file)
noise_values = noise_data["max"] # choose to present the max, can also present the mean with 'mean'

# Add a column to the table with header "Exp."

table["Exp."] = [f"${value:.6f}$" for value in noise_values]

# Get the latex code:
table_header = "& \multicolumn{2}{ c }{GW170817} & \multicolumn{2}{ c }{GW190425} & \\\\\n"
table_str = my_to_latex(table, extra_table_header=table_header)

print(table_str)


\begin{tabular}{l  c  c  c  c  c}
\hline\hline
& \multicolumn{2}{ c }{GW170817} & \multicolumn{2}{ c }{GW190425} & \\

Parameter & TF2 & NRTv2 & TF2 & NRTv2 & Exp. \\

$\mathcal{M}$ & $0.001725$ & $0.000516$ & $0.003557$ & $0.001884$ & $0.000261$ \\
$q$ & $0.005212$ & $0.007894$ & $0.004837$ & $0.003883$ & $0.000242$ \\
$\chi_1$ & $0.005633$ & $0.004301$ & $0.002794$ & $0.001904$ & $0.000491$ \\
$\chi_2$ & $0.003030$ & $0.002671$ & $0.002416$ & $0.001400$ & $0.000247$ \\
$\Lambda_1$ & $0.001062$ & $0.002208$ & $0.008556$ & $0.000652$ & $0.000363$ \\
$\Lambda_2$ & $0.000559$ & $0.002186$ & $0.005808$ & $0.001450$ & $0.000477$ \\
$d_L$ & $0.001544$ & $\bm{0.01847}$ & $0.001273$ & $0.009157$ & $0.000969$ \\
$\phi_c$ & $0.003500$ & $0.010714$ & $0.003338$ & $0.007874$ & $0.000281$ \\
$\cos \iota$ & $0.001615$ & $0.012851$ & $0.006400$ & $0.009502$ & $0.000289$ \\
$\psi$ & $0.004048$ & $0.011036$ & $0.001516$ & $0.005820$ & $0.000203$ \\
$\alpha$ & $\bm{0.014008}$ & $0.001258$ & $\bm{0.009